In [2]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

In [3]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

100.1%

Extracting ./mnist/MNIST\raw\train-images-idx3-ubyte.gz to ./mnist/MNIST\raw


113.5%

Extracting ./mnist/MNIST\raw\train-labels-idx1-ubyte.gz to ./mnist/MNIST\raw


100.4%

Extracting ./mnist/MNIST\raw\t10k-images-idx3-ubyte.gz to ./mnist/MNIST\raw


180.4%

Extracting ./mnist/MNIST\raw\t10k-labels-idx1-ubyte.gz to ./mnist/MNIST\raw
Processing...
Done!


In [4]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 1,28x28
        self.conv1=nn.Conv2d(1,10,5) # 10, 24x24
        self.conv2=nn.Conv2d(10,20,3) # 128, 10x10
        self.fc1 = nn.Linear(20*10*10,500)
        self.fc2 = nn.Linear(500,10)
    def forward(self,x):
        in_size = x.size(0)
        out = self.conv1(x) #24
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2)  #12
        out = self.conv2(out) #10
        out = F.relu(out)
        out = out.view(in_size,-1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.log_softmax(out,dim=1)
        return out

In [10]:

import torch.optim as optim
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 让torch判断是否使用GPU，建议使用GPU环境，因为会快很多
model = ConvNet().to(DEVICE)
optimizer = optim.Adam(model.parameters())

In [11]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%30 == 0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [12]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [14]:
import torch.nn.functional as F
for epoch in range(1, NUM_EPOCHS + 1):
    train(model, DEVICE, train_loader, optimizer, epoch)
    test(model, DEVICE, test_loader)

Train Epoch: 1 [3712/60000 (6%)]	Loss: 0.509756
Train Epoch: 1 [7552/60000 (13%)]	Loss: 0.418435
Train Epoch: 1 [11392/60000 (19%)]	Loss: 0.303486
Train Epoch: 1 [15232/60000 (25%)]	Loss: 0.149126
Train Epoch: 1 [19072/60000 (32%)]	Loss: 0.203605
Train Epoch: 1 [22912/60000 (38%)]	Loss: 0.056012
Train Epoch: 1 [26752/60000 (45%)]	Loss: 0.136871
Train Epoch: 1 [30592/60000 (51%)]	Loss: 0.036254
Train Epoch: 1 [34432/60000 (57%)]	Loss: 0.201549
Train Epoch: 1 [38272/60000 (64%)]	Loss: 0.083180
Train Epoch: 1 [42112/60000 (70%)]	Loss: 0.073207
Train Epoch: 1 [45952/60000 (77%)]	Loss: 0.151191
Train Epoch: 1 [49792/60000 (83%)]	Loss: 0.077862
Train Epoch: 1 [53632/60000 (90%)]	Loss: 0.079721
Train Epoch: 1 [57472/60000 (96%)]	Loss: 0.025083

Test set: Average loss: 0.0595, Accuracy: 9795/10000 (98%)

Train Epoch: 2 [3712/60000 (6%)]	Loss: 0.081630
Train Epoch: 2 [7552/60000 (13%)]	Loss: 0.132877
Train Epoch: 2 [11392/60000 (19%)]	Loss: 0.063350
Train Epoch: 2 [15232/60000 (25%)]	Loss: 0.04